In [1]:
from typing import Dict
import json
from langgraph.graph import MessagesState
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
import uvicorn
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_together import ChatTogether
from pydantic import Field,BaseModel

In [2]:
import psycopg2
def fetch_balances():
    try:
        connection = psycopg2.connect(
            dbname="bankapp",
            user="postgres",
            password="manu",
            host="localhost",
            port=5432
        )
        cursor = connection.cursor()
        fetch_balance_query = "SELECT * FROM users WHERE name = %s"

        cursor.execute(fetch_balance_query, ('Grace Lee',))
        sender_details = cursor.fetchone()

        if sender_details is None:
            return {'messages': "User 'Grace Lee' not found"}

        return {'details':sender_details }
    
    except Exception as e:
        return {'messages': f"An error occurred while fetching balances: {e}"}
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()

result = fetch_balances()
print(result)



{'details': (7, 'Grace Lee', '159 Willow Way, GA', 8444556677, 'TD Bank', 23820)}


In [3]:
import psycopg2
from datetime import date  

def insert_transaction(sen_name, rec_name, amount):
    try:
     
        connection = psycopg2.connect(
            dbname="userdetails",
            user="postgres",
            password="manu",
            host="localhost",
            port=5432
        )
        cursor = connection.cursor()
        

        transaction_date = date.today()


        insert_query = """
        INSERT INTO transaction (sen_name, rec_name, amount, date) 
        VALUES (%s, %s, %s, %s)
        """
        
 
        cursor.execute(insert_query, (sen_name, rec_name, amount, transaction_date))
        

        connection.commit()
        
        return {'message': "Transaction inserted successfully"}
    
    except psycopg2.Error as e:
        return {'message': f"Database error: {e}"}
    
    except Exception as e:
        return {'message': f"An unexpected error occurred: {e}"}
    
    finally:

        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'connection' in locals() and connection:
            connection.close()

result = insert_transaction('Grace Lee', 'John Doe', 100.0)
print(result)


{'message': 'Transaction inserted successfully'}


In [4]:
def fetch_balances1():
    try:
        connection = psycopg2.connect(
            dbname="userdetails",
            user="postgres",
            password="manu",
            host="localhost",
            port=5432
        )
        cursor = connection.cursor()
        fetch_balance_query = "SELECT * FROM transaction WHERE sen_name = %s"

        cursor.execute(fetch_balance_query, ('Grace Lee',))
        sender_details =cursor.fetchall()

        if sender_details is None:
            return {'messages': "User 'Grace Lee' not found"}

        return {'details':sender_details }
    
    except Exception as e:
        return {'messages': f"An error occurred while fetching details: {e}"}
    
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()


In [5]:
result=fetch_balances1()

In [6]:
result

{'details': [('Grace Lee', 'John Doe', 100.0, datetime.date(2025, 2, 22)),
  ('Grace Lee', 'Eve Adams', 1.0, datetime.date(2025, 2, 22)),
  ('Grace Lee', 'Eve Adams', 1.0, datetime.date(2025, 2, 22)),
  ('Grace Lee', 'Bob Johnson', 20.0, datetime.date(2025, 2, 22)),
  ('Grace Lee', 'Bob Johnson', 30.0, datetime.date(2025, 2, 22)),
  ('Grace Lee', 'John Doe', 100.0, datetime.date(2025, 3, 8))]}

In [7]:
new_list=[]
for datas in result['details']:
    details={
        "name":datas[1],
        "amount":datas[2],
        "date":datas[3]
    }
    new_list.append(details)    

In [8]:
new_list

[{'name': 'John Doe', 'amount': 100.0, 'date': datetime.date(2025, 2, 22)},
 {'name': 'Eve Adams', 'amount': 1.0, 'date': datetime.date(2025, 2, 22)},
 {'name': 'Eve Adams', 'amount': 1.0, 'date': datetime.date(2025, 2, 22)},
 {'name': 'Bob Johnson', 'amount': 20.0, 'date': datetime.date(2025, 2, 22)},
 {'name': 'Bob Johnson', 'amount': 30.0, 'date': datetime.date(2025, 2, 22)},
 {'name': 'John Doe', 'amount': 100.0, 'date': datetime.date(2025, 3, 8)}]